In [1]:
using Chemfiles

In [2]:
function displaceAA(in_frm, aa, aa_3, in_vec)
    # Preparo variables
     in_top = Topology(in_frm)
    natoms = convert(Int64, size(in_top))
     in_xyz = positions(in_frm)

    # Determino orden de residuos (hay q actualizar el Julia Chemfiles)
    tmp = Array{Int64}(aa)
    ids = Array{Int64}(aa)
    [ ids[i+1] = convert(Int64, id((Residue(in_top, i)))) for i = 0:aa-1 ]
     idx = sortperm(ids)
    # Determino el nro de atomos de c/ aminoácido
    [ tmp[i+1] = size(Residue(in_top, i)) for i = 0:aa-1 ]
     natom_aa = tmp[idx]

    # Paso el vector columna de tamaño 1xaa_3 a 3xaa
     vector = reshape(in_vec, 3, aa)
    # Adapto el vector p/ darle la misma forma q la matriz de coordenadas
    sum_mat = Array{Float64}(3, natoms)
    cursor = 0
    for i = 1:aa
        if i == 1
            sum_mat[:, 1:natom_aa[i]] = repmat(vector[:, 1], 1, natom_aa[i])
            cursor = natom_aa[i]
            continue
        end
         rango = collect(cursor+1:cursor + natom_aa[i])
        sum_mat[:, rango] = repmat(vector[:, i], 1, natom_aa[i])
        cursor += natom_aa[i]
    end

    # Listo, ahora puedo mover el pdb
    out_frm = deepcopy(in_frm)
    out_xyz = positions(out_frm)
    out_xyz  = in_xyz + sum_mat

    writedlm("a", out_xyz)
    writedlm("b", in_xyz)
    writedlm("c", positions(out_frm))

    return out_frm
end

displaceAA (generic function with 1 method)

In [3]:
in_pdb_filename = "1M14_A.pdb"
modes_filename = "modos_1M14_A"
suffix = "as"
mul = 50

50

In [4]:
# Read PDB
 in_trj = Trajectory(in_pdb_filename)
 in_frm = read(in_trj)
 in_top = Topology(in_frm)
 aa = convert(Int64, count_residues(in_top))
 aa_3 = aa * 3

831

In [5]:
in_modes = Array{Float64, 2}(aa_3, aa)
if (false)
# Modos de PCA Amber
    try
        in_modes = read_ptraj_modes(modes_filename, aa_3, true)[1][:, index]
    catch
        try
            in_modes = read_ptraj_modes(modes_filename, natom_xyz,
                true)[1][:, index]
        end
    end
else
    try
        in_modes = convert(Array{Float64, 2}, readdlm(modes_filename))
    catch e
        error(string("\n\n", modes_filename, " could not be found."))
    end
end

# Check input modes
if size(in_modes)[1] != aa_3
# Asumo q el modo es de Calpha y está ordenado en una columna
    error(string("\n\n", " Input PDB has ", aa, " residues, so input ",
        "modes should be ", aa_3, " long."))
end

In [ ]:
# Ahora desplazo
#for i = 1:size(in_modes)[2]
i = 1
    # Escalo vector
     modo = in_modes[:, i] .* mul
    # Desplazo
     out_frm = displaceAA(in_frm, aa, aa_3, modo);


In [6]:
in_vec =  in_modes[:, 1] .* mul
    # Preparo variables
     in_top = Topology(in_frm)
    natoms = convert(Int64, size(in_top))
     in_xyz = positions(in_frm)

    # Determino orden de residuos (hay q actualizar el Julia Chemfiles)
    tmp = Array{Int64}(aa)
    ids = Array{Int64}(aa)
    [ ids[i+1] = convert(Int64, id((Residue(in_top, i)))) for i = 0:aa-1 ]
     idx = sortperm(ids)
    # Determino el nro de atomos de c/ aminoácido
    [ tmp[i+1] = size(Residue(in_top, i)) for i = 0:aa-1 ]
     natom_aa = tmp[idx]

    # Paso el vector columna de tamaño 1xaa_3 a 3xaa
     vector = reshape(in_vec, 3, aa)
    # Adapto el vector p/ darle la misma forma q la matriz de coordenadas
    sum_mat = Array{Float64}(3, natoms)
    cursor = 0
    for i = 1:aa
        if i == 1
            sum_mat[:, 1:natom_aa[i]] = repmat(vector[:, 1], 1, natom_aa[i])
            cursor = natom_aa[i]
            continue
        end
         rango = collect(cursor+1:cursor + natom_aa[i])
        sum_mat[:, rango] = repmat(vector[:, i], 1, natom_aa[i])
        cursor += natom_aa[i]
    end

    # Listo, ahora puedo mover el pdb
    out_frm = deepcopy(in_frm)
    out_xyz = positions(out_frm)
    out_xyz  = in_xyz + sum_mat

3×2224 Array{Float64,2}:
 39.6172  39.1753  38.4932  37.3122  …  10.9602  11.3843  11.4213   6.76125
 -8.0675  -9.3565  -9.1805  -8.8405     18.8816  17.5345  19.9326  20.0875 
 64.775   64.1941  62.8811  62.829      68.3034  67.8495  67.3245  67.9335 

In [10]:
as = positions(out_frm) .+ 1.0

3×2224 Array{Float64,2}:
 41.926  41.484  40.802  39.621  40.482  …  11.716  12.14   12.177   7.517
 -5.34   -6.629  -6.453  -6.113  -7.33      19.552  18.205  20.603  20.758
 64.315  63.734  62.421  62.369  64.668     68.948  68.494  67.969  68.578

In [16]:
as

3×2224 Array{Float64,2}:
 41.926  41.484  40.802  39.621  40.482  …  11.716  12.14   12.177   7.517
 -5.34   -6.629  -6.453  -6.113  -7.33      19.552  18.205  20.603  20.758
 64.315  63.734  62.421  62.369  64.668     68.948  68.494  67.969  68.578

In [21]:
positions(out_frm)

3×2224 Array{Float64,2}:
  1.0    40.484  39.802  38.621  39.482  …  10.716  11.14   11.177   6.517
 -6.34   -7.629  -7.453  -7.113  -8.33      18.552  17.205  19.603  19.758
 63.315  62.734  61.421  61.369  63.668     67.948  67.494  66.969  67.578

In [20]:
pos[1, 1] = 1.0

1.0

In [15]:
out_frm.positions()

LoadError: [91mtype Frame has no field positions[39m

In [ ]:
    # Y guardo
    out_trj = Trajectory(string(i, "_", suffix), 'w')
    write(out_trj, out_frm)